# Stochastic simulation assignment 2
## Peter Voerman and Nick van Santen
### 11749547 and ...

In [14]:
import math
import numpy as np
import random
import simpy

In [21]:

class Server():

    def __init__(self, env, n_machines):
        self.env = env
        self.machine = simpy.Resource(env, n_machines)

    def run_task(self, task):
        yield self.env.timeout(task.get_task_duration())

    @property
    def queue_length(self):
        return len(self.machine.queue)

class Task():

    def __init__(self, name, duration):
        self.name = name
        self.duration = duration

    def get_task_duration(self):
        return generate_random_markov(self.duration)

    def __str__(self):
        return f"{self.name}"

def process_task(env, task, server, data):

    #print(f"{task} added to server at {env.now}. Queue length: {server.queue_length}")
    time_at_queue = env.now
    with server.machine.request() as request:
        yield request

        #print(f"{task} is being processed at {env.now}")
        time_start_process = env.now
        yield env.process(server.run_task(task))
        #print(f"{task} is completed at {env.now}")
        time_end_process = env.now

    data["wait_times"].append(time_start_process - time_at_queue)
    data["process_times"].append(time_end_process - time_start_process)

def setup(env, init, data):
    INIT_TASKS = init[0]
    N_MACHINES = init[1]

    MARKOV_TASK_DURATION = init[2]
    MARKOV_TASK_ARRIVAL = init[3]

    server = Server(env, N_MACHINES)

    for i in range(INIT_TASKS):
        env.process(process_task(env, Task(f"Task {i}", MARKOV_TASK_DURATION), server, data))

    while True:
        yield env.timeout(generate_random_markov(MARKOV_TASK_ARRIVAL))
        i += 1
        env.process(process_task(env, Task(f"Task {i}", MARKOV_TASK_DURATION), server, data))

def generate_random_markov(lamda):
    """
    Markov CDF: y = 1 - e^(-lamda t)
    The y value has a range of 0 to 1, which we can sample.
    Thus we can obtain a random t value by sampling y

    t = - ln(1 - y) / lamda
    """

    r = random.random()
    return -math.log(1 - r) / lamda

def generate_long_tail():
    """
    Generate a random number from a distribution where 75% of the jobs have an exponential distribution
    with an average service time of 1.0 and the remaining 25% an exponential distribution
    with an average service time of 5.0.

    The average of the exponential distribution is 1 / lamda. Thus lamda = 1 / expectation.
    """

    r = random.random()
    
    if r < 0.75:
        lamda = 1 / 1
    else:
        lamda = 1 / 5
    
    return generate_random_markov(lamda)

In [24]:
def run(init):
    data = {
    "wait_times": [],
    "process_times": [],
    }
    env = simpy.Environment()
    env.process(setup(env, init, data))
    env.run(until=MAX_SIM_TIME)
    
    


    print(f"Avg wait time: {np.mean(data['wait_times'])}")
    print(f"Avg process time: {np.mean(data['process_times'])}")

In [27]:
INIT_TASKS = 4
N_MACHINES = 1
MAX_SIM_TIME = 100000

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 0.9



init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL]

run(init)
run(init)
run(init)

Avg wait time: 8.932163942370629
Avg process time: 1.0030364759328807
Avg wait time: 9.409788821674526
Avg process time: 1.0041316021086497
Avg wait time: 8.658081400265822
Avg process time: 0.9962312890385785


In [28]:
INIT_TASKS = 4
N_MACHINES = 2
MAX_SIM_TIME = 100000

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 1.8

# data = {
#     "wait_times": [],
#     "process_times": [],
# }

init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL]

run(init)

Avg wait time: 4.425313280392082
Avg process time: 1.0019738289243107
